In [ ]:
"""
Prepare some masks for ORCA1
"""

In [94]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

In [15]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [2]:
inputpath = '/data/cburgard/PREPARE_FORCING/'

In [71]:
data_bathy = xr.open_dataset(inputpath + 'eORCA1.4.2_ModifStraits_domain_ALLcav_KH.nc').squeeze()

In [70]:
data_bathy['bathymetry_isf'].squeeze().plot()

In [32]:
mask_on_NEMO_grid = xr.open_dataset(inputpath + 'Mask_Iceshelf_IMBIE_NEMO1.nc')
mask_orig = xr.open_dataset(inputpath + 'Mask_Iceshelf_IMBIE.nc')

In [18]:
isf_mask = mask_on_NEMO_grid['Iceshelf_extrap'].where(data_bathy['isf_draft'] > 0).squeeze()

CHECK WHICH ICE SHELVES TO OPEN

In [64]:
thres = 5
isf_ok_list = []
for kisf in range(1,134):
    if (isf_mask == kisf).any():
        kisf_mask = isf_mask.where(isf_mask == kisf, drop=True)

        kisf_x = len(kisf_mask.x)
        kisf_y = len(kisf_mask.y)
        if (kisf_x >= thres) & (kisf_y >= thres): # this could be improved as it favors also ice shelves that "turn" like George VI
            isf_ok_list.append(kisf)
        

In [65]:
for kisf in isf_ok_list: 
    print(kisf, mask_orig['NAME'].sel(ID=kisf).values)

2 Fimbul
9 Baudouin
21 Amery
66 Ross_East
67 Ross_West
75 Sulzberger
82 Getz
99 George_VI
109 Wilkins
117 LarsenC
124 Ronne
125 Filchner
127 Brunt_Stancomb
128 Riiser-Larsen


In [66]:
for kisf in isf_ok_list: 
    plt.figure()
    (isf_mask == kisf).plot()
    plt.title(mask_orig['NAME'].sel(ID=kisf).values)


MAKE A CLEAN MASK

In [83]:
mask_0_1_2 = isf_mask * 2
mask_0_1_2 = mask_0_1_2.where(data_bathy['bathymetry_isf'] != 0, 400) # land
mask_0_1_2 = mask_0_1_2.where(data_bathy['bathymetry_isf'] == 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(data_bathy['isf_draft'] == 0, 200) # ice shelf

In [137]:
# mask putting the small ice shelves to land
mask_open = isf_mask.copy()
mask_0_1_2_corr = mask_0_1_2.copy()

for kisf in range(1,134):
    if kisf not in isf_ok_list:
        mask_open = mask_open.where(isf_mask != kisf, np.nan)
        mask_0_1_2_corr = mask_0_1_2_corr.where(isf_mask != kisf, 400)

DEFINE THE ZONE AT THE ICE SHELF FRONT

In [138]:
mask_open2 = isf_mask.copy()
mask_0_1_2_corr2 = mask_0_1_2.copy()

for kisf in range(1,134):
    if kisf in isf_ok_list:
        mask_open2 = mask_open2.where(isf_mask != kisf, np.nan)
        mask_0_1_2_corr2 = mask_0_1_2_corr2.where(isf_mask != kisf, 400)

In [139]:
is_mask0 = mask_open.where(np.isnan(mask_open)==False,0)
# set all ice shelves to 3
mask_front = mask_0_1_2_corr2.where((mask_0_1_2_corr2 == 0) | (mask_0_1_2_corr2 == 400), 300).copy()

In [140]:
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front.shift(x=-1)-mask_front)!=300,500)
mask_front = mask_front.where((mask_front.shift(x=1)-mask_front)!=300,500)
mask_front = mask_front.where((mask_front.shift(y=-1)-mask_front)!=300,500)
mask_front = mask_front.where((mask_front.shift(y=1)-mask_front)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,mask_on_NEMO_grid['Iceshelf_extrap'])

In [135]:
for kisf in range(1,134):
    if kisf not in isf_ok_list:
        if ((isf_mask == kisf).any()) & (kisf != 105):
            print(kisf, mask_orig['NAME'].sel(ID=kisf).values)
            all_mask = ((isf_mask == kisf).astype(int)*2) + ((mask_front == kisf).astype(int)*1)
            plt.figure()
            all_mask.where(all_mask > 0, drop=True).plot() 

1 Jelbart
4 Vigrid
5 Nivl
7 Lazarev
8 Borchgrevink
11 Prince_Harald
24 West
26 Shackleton
27 Tracy_Tremenchus
29 Conger_Glenzer
33 Totten
43 Mertz
47 Cook
53 Rennick
59 Mariner
63 Nansen
64 Drygalski
77 Nickerson
83 Dotson/Philbin_Inlet
85 Thwaites
86 Pine_Island


/tmp/ipykernel_4043461/1867761176.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


87 Walgreen_Coast
88 Cosgrove
90 Abbot
92 Venable
98 Stange
101 Bach
116 LarsenB
118 LarsenD
130 Quar
132 Ekstrom
133 Atka


In [ ]:
plt.close('all')

PUT IN ONE NETCDF

In [141]:
final_ds = xr.merge([mask_0_1_2_corr.rename('mask_0_200_400'),
             mask_open.rename('open_isf'),
             mask_front.rename('front_closed_isf'),
             mask_orig['NAME']])

In [ ]:
final_ds.to_dataset(to_be_filled)